In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

# enumerate happiness

# Parameters
N1, N2 = 64, 64
probability = 0.5  # Very crowded!
ratio = 0.6
#initial conditions of zero
neighborhood = np.zeros((N1,N2), dtype=int)

# Site percolation
def initialize_schelling_grid(neighborhood, probability, ratio):
    empty = []
    active = []
    N1 = neighborhood.shape[0]
    N2 = neighborhood.shape[1]
    for i in range(N1): #for cell in every row
        for j in range(N2): #and every column
            die = random.uniform(0, 1)
            if die < probability:
              die_type = random.uniform(0,1)
              if die_type < ratio:
                neighborhood[(i,j)] = 1
                active.append((i,j))
              else:
                neighborhood[(i,j)] = np.random.randint(2, 5) # 3 = Sweet Grandma that bakes everyone cookies 4 = Drummer that plays loud, late, early, and sucks
                active.append((i,j))
            else:
              neighborhood[(i,j)] = 0
              empty.append((i,j))


    return neighborhood, empty, active
    
neighborhood, empty, active = initialize_schelling_grid(neighborhood,probability, ratio)

def check_happiness(neighborhood, i, j, agent_type=None):
    """
    Check happiness at position (i,j) for a given agent type.
    If agent_type is None, use the current occupant's type.
    """
    if agent_type is None:
        current_type = neighborhood[i, j]
    else:
        current_type = agent_type
    
    # Can't check happiness for empty cells
    if current_type == 0:
        return 0
    
    N1, N2 = neighborhood.shape
    
    # Get all valid neighbors
    neighbors = []
    
    # Define the 8 possible neighbor directions (including diagonals)
    directions = [
        (-1, -1), (-1, 0), (-1, 1),  # top row
        (0, -1),           (0, 1),    # same row (left and right)
        (1, -1),  (1, 0),  (1, 1)     # bottom row
    ]
    
    for di, dj in directions:
        ni, nj = i + di, j + dj
        # Check if neighbor is within bounds
        if 0 <= ni < N1 and 0 <= nj < N2:
            neighbors.append(neighborhood[ni, nj])
    
    # Count neighbors of the same type (excluding empty cells)
    same_type = sum(1 for n in neighbors if n == current_type or n == 3) # 3 = Sweet Grandma that bakes everyone cookies
    different_type = sum(1 for n in neighbors if n != current_type and n != 0 and n != 3) # 4 = Drummer that plays loud, late, early, and sucks

    contains_sweet_grandma = any(n == 3 for n in neighbors)
    contains_drummer = any(n == 4 for n in neighbors)
    
    # Only count non-empty neighbors
    non_empty_neighbors = same_type + different_type
    
    # Handle edge case of no neighbors
    if non_empty_neighbors == 0:
        return 1  # Happy if isolated (or return 0 if you prefer)
    
    current_happiness = same_type / non_empty_neighbors >= 0.5
    if contains_sweet_grandma:
        current_happiness += 0.22
    if contains_drummer:
        current_happiness -= .25
    # Happy if at least n% of non-empty neighbors are same type
    if current_happiness >= 0.5:
        return 1
    else:
        return -1


def run_schelling(neighborhood, empty, active):
    """
    Run one step of the Schelling model.
    Unhappy agents move to random empty cells.
    """
    N1, N2 = neighborhood.shape
    
    # Iterate over a copy of active to avoid modification issues
    for cell in list(active):
        i, j = cell
        agent_type = neighborhood[i, j]
        
        # Check current happiness
        happiness = check_happiness(neighborhood, i, j)
        
        if happiness < 1 and len(empty) > 0:
            # Agent is unhappy - try to move
            empty_idx = random.randint(0, len(empty) - 1)
            potential_i, potential_j = empty[empty_idx]
            
            # Check what happiness would be at the new location
            # (passing agent_type since that cell is currently empty)
            potential_happiness = check_happiness(neighborhood, potential_i, potential_j, agent_type)
            
            # Move if new location would make agent happy
            if potential_happiness >= 1:
                # Perform the move
                neighborhood[potential_i, potential_j] = agent_type
                neighborhood[i, j] = 0  # Mark old location as empty
                
                # Update lists
                empty.pop(empty_idx)  # Remove new location from empty
                empty.append((i, j))  # Add old location to empty
                active.remove(cell)   # Remove old location from active
                active.append((potential_i, potential_j))  # Add new location to active
    return neighborhood, empty, active

def run_schelling_single_move(neighborhood, empty, active):
    """
    Run one step of the Schelling model.
    Unhappy agents move to random empty cells.
    """
    N1, N2 = neighborhood.shape
    
    # Iterate over a copy of active to avoid modification issues
    for cell in list(active):
        i, j = cell
        agent_type = neighborhood[i, j]
        
        # Check current happiness
        happiness = check_happiness(neighborhood, i, j)
        
        if happiness < 1 and len(empty) > 0:
            # Agent is unhappy - try to move
            empty_idx = random.randint(0, len(empty) - 1)
            potential_i, potential_j = empty[empty_idx]
            
            # Check what happiness would be at the new location
            # (passing agent_type since that cell is currently empty)
            potential_happiness = check_happiness(neighborhood, potential_i, potential_j, agent_type)
            
            # Move if new location would make agent happy
            if potential_happiness >= 1:
                # Perform the move
                neighborhood[potential_i, potential_j] = agent_type
                neighborhood[i, j] = 0  # Mark old location as empty
                
                # Update lists
                empty.pop(empty_idx)  # Remove new location from empty
                empty.append((i, j))  # Add old location to empty
                active.remove(cell)   # Remove old location from active
                active.append((potential_i, potential_j))  # Add new location to active
                #print(f'Old location is {i},{j}')
                #print(f'New location is {potential_i},{potential_j}')
                break
    return neighborhood, empty, active


import matplotlib.pyplot as plt

def visualize_grid(neighborhood, step=0):
    plt.figure(figsize=(8, 8))
    # Create color map: 0=white, 1=blue, 2=red
    colors = np.zeros((*neighborhood.shape, 3))
    colors[neighborhood == 1] = [0, 0, 1]  # Blue
    colors[neighborhood == 2] = [1, 0, 0]  # Red
    colors[neighborhood == 0] = [1, 1, 1]  # White
    
    plt.imshow(colors, interpolation='nearest')
    plt.title(f'Step {step}')
    plt.axis('off')
    plt.show()

def visualize_grid_and_happiness(neighborhood, step=0):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
    colors = np.zeros((*neighborhood.shape, 3))
    colors[neighborhood == 1] = [0, 0, 1]  # Blue
    colors[neighborhood == 2] = [1, 0, 0]  # Red
    colors[neighborhood == 0] = [1, 1, 1]  # White
    colors[neighborhood == 3] = [0, 1, 0]  # Green
    colors[neighborhood == 4] = [1, 1, 0]  # Yellow
    
    ax1.imshow(colors, interpolation='nearest')
    fig.suptitle(f'Step {step}')
    #ax1.axis('off')
    ax2.plot(happiness_history)
    plt.show()

happiness_history = []
visualize_grid_and_happiness(neighborhood, 0)
for step in range(200):
    total_happiness = []
    for item in active:
        happiness_cell = item
        total_happiness.append(check_happiness(neighborhood,happiness_cell[0],happiness_cell[1]))
    #print(total_happiness.count(1))
    happiness_ratio = 1 - (total_happiness.count(-1)) / (total_happiness.count(1))
    #print(f'Happiness Ratio is {happiness_ratio}')
    happiness_history.append(happiness_ratio)
    #run_schelling(neighborhood, empty, active)
    run_schelling_single_move(neighborhood,empty,active)
    if happiness_ratio == 1:
        visualize_grid_and_happiness(neighborhood, step)
        print(f'Finished running in {step} moves')
        break
    if step % 5 == 4:  # Show every 5th step
        #visualize_grid(neighborhood, step + 1)
        visualize_grid_and_happiness(neighborhood, step + 1)


NameError: name 'empty' is not defined

In [43]:
# from matplotlib.colors import ListedColormap
# cmap = ListedColormap(['white', 'red', 'blue'])
# fig, ax1 = plt.subplots()
# ax1.set_ylim(0,N1)
# ax1.set_xlim(0,N2)
# ax1.set_aspect('equal')
# ax1.imshow((grid), cmap=cmap, origin = 'lower', vmin=0, vmax=2)
# plt.show()